In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [3]:
# read the 'Mother Report' excel file
mother_report = pd.read_excel('./input/MR.xlsx')

In [4]:
mother_report = mother_report.drop(mother_report.columns[-1:-13:-1], axis = 1)
mother_report = mother_report.drop(['Eligible Couple Number', 'Other Religion', 'Vdrl Test Date', 'Hiv Test Date'], axis = 1)
mother_report = mother_report.drop(mother_report.columns[-1:-8:-1], axis = 1)

In [5]:
# EDA. To find out the amount of Missing Values in each of the columns that start with 'ANC1'.
# Same has benn done for other columns too, but not reproduced here.
mother_report[mother_report.columns[mother_report.columns.str.startswith('ANC1')]].isnull().sum()

ANC1 Date                    66
ANC1 Weight                  66
ANC1 BP Sys                  66
ANC1 BP Dia                  66
ANC1 HB                      66
ANC1 Urine Albumin           72
ANC1 Urine Sugar             72
ANC1 Blood Sugar              0
ANC1 Fundal Height           66
ANC1 Fetal Movements         66
ANC1 Fetal Position          66
ANC1 Fetal Heart Rate      4845
ANC1 Albendazole           2047
ANC1 Albendazole Status    4270
ANC1 IFA Date                88
ANC1 IFA Status              85
ANC1 High Risk Symptoms     661
ANC1 Referral Location     4321
dtype: int64

In [6]:
# Read in the 'Delivery_Outcome_report file 
delivery_outcome = pd.read_excel('./input/DDR.xlsx')

In [7]:
# Set the entries of outcome variable to 'Abortion' and 'Miscarriage' when 'No. Abortion' > 0 and 'No. Miscarriage' > 0
delivery_outcome.loc[(delivery_outcome['No. Abortion'] > 0)  , ['Live Birth Or Still Birth']] = 'Abortion'
delivery_outcome.loc[(delivery_outcome['No. Miscarriage'] > 0) , ['Live Birth Or Still Birth']] = 'Miscarriage'

In [8]:
#Filter out the entries that don't have a value in the 'Live Birth Or Still Birth' field and rename it as delivery_1
delivery_1 = delivery_outcome[~delivery_outcome['Live Birth Or Still Birth'].isnull()]


In [9]:
#Read in the other file with outcomes.
delivery_dhan = pd.read_excel('./input/DDCD.xlsx')


In [10]:

#Set the value for entries where the outcome variable is not missing AND where the infant has died, to 'Infant Death' and name the 
#resulting dataframe as delivery_2
delivery_dhan.loc[((~delivery_dhan['Delivery outcome'].isnull()) & (delivery_dhan['Infant Death'] == 'Yes')), 'Delivery outcome'] = 'Infant Death'
delivery_2 = delivery_dhan.loc[((~delivery_dhan['Delivery outcome'].isnull()) & (delivery_dhan['Infant Death'] == 'Yes'))]


In [11]:

#make sure that it only contains entries whose outcome values are not missing
delivery_2 = delivery_2[~delivery_2['Delivery outcome'].isnull()]

In [12]:

#Now we've to make a single dataframe(df) having data from both 'Delivery_Outcome_report and Dhanwant_Delivery
#Make a temporary df having relevant columns from delivery_1
merge = pd.DataFrame()
merge['ID'] = delivery_1['Mother ID']
merge['Outcome'] = delivery_1['Live Birth Or Still Birth']


In [13]:

#Another temporary df that takes in all the non-common patients from delivery_1 and delivery_2
tmp = delivery_2[~delivery_2['Patient ID'].isin(delivery_1['Mother ID'])][['Patient ID', 'Delivery outcome']]


In [14]:

#Now we concatenate the non-common entries from both delivery_1 and delivery_2 to the 'merge' dataframe containing delivery_1 entries
#First we rename the columns in 'tmp' so that they are concatenated along the same columns names
tmp = tmp.reset_index().drop('index', axis = 1).rename(columns = {'Patient ID' : 'ID', 'Delivery outcome' : 'Outcome'})
#concatenate along the columns( add new rows with same columns )
merged = pd.concat((merge, tmp), axis = 0, ignore_index = True)

In [15]:

#Now merge it with the 'Mother_report' df so that we have outcomes for the data we need to train on
df = pd.merge(mother_report, merged, left_on = 'Registration ID', right_on = 'ID').drop('ID',axis = 1)



In [16]:
#Make sure that all outcomes ('Live Birth', 'Still Birth', 'Abortion', 'Infant Death', 'Miscarriage') are present in the final df
df['Outcome'].unique()



array(['Live Birth', 'Still Birth', 'Abortion', 'Infant Death',
       'Miscarriage'], dtype=object)

In [17]:
#Again see how many missing values are left in the new df
df[df.columns[df.columns.str.startswith('ANC1')]].isnull().sum()


ANC1 Date                    29
ANC1 Weight                  29
ANC1 BP Sys                  29
ANC1 BP Dia                  29
ANC1 HB                      29
ANC1 Urine Albumin           31
ANC1 Urine Sugar             31
ANC1 Blood Sugar              0
ANC1 Fundal Height           29
ANC1 Fetal Movements         29
ANC1 Fetal Position          29
ANC1 Fetal Heart Rate      1848
ANC1 Albendazole            746
ANC1 Albendazole Status    1658
ANC1 IFA Date                38
ANC1 IFA Status              36
ANC1 High Risk Symptoms     287
ANC1 Referral Location     1674
dtype: int64

In [18]:
df2 = df.drop(['LMP Date', 'Religion', 'Bank Name', 'Blood Group', 'Drop Out',
         'Past Illnesses', 'Penultimate Preg Outcomes', 
         'Prior Pregnancy Complications', 'Block', 'Camp Name', 
         'Active', 'Returning', 
         'ANC1 IFA Date', 'ANC2 IFA Date', 'ANC3 IFA Date', 'ANC4 IFA Date',
         'ANC1 Albendazole Status', 'ANC2 Albendazole Status', 'ANC3 Albendazole Status', 'ANC4 Albendazole Status',
         'ANC1 Referral Location', 'ANC2 Referral Location', 'ANC3 Referral Location', 'ANC4 Referral Location',
        'ANC1 Albendazole', 'ANC2 Albendazole', 'ANC3 Albendazole', 'ANC4 Albendazole', 
         'ANC1 IFA Status', 'ANC2 IFA Status', 'ANC3 IFA Status', 'ANC4 IFA Status', 
         'ANC1 Urine Sugar', 'ANC2 Urine Albumin', 'ANC2 Urine Sugar',"ANC1 Urine Albumin ",
        'ANC3 Urine Albumin', 'ANC3 Urine Sugar', 'ANC4 Urine Albumin', 'ANC4 Urine Sugar'], axis = 1)
		

In [19]:

# Since the sting values in the data such as 'No Equipment' in the variable 'ANC Heart Rate' have been misspelled 
#and expressed in different languages, make it into a single value, 'NO Equipment'
df2[df2.columns[df2.columns.str.contains('Heart Rate')]].loc [((df2['ANC1 Fetal Heart Rate'].str.contains('quipm')) | (df2['ANC2 Fetal Heart Rate'].str.contains('quipm'))
                         | (df2['ANC3 Fetal Heart Rate'].str.contains('quipm')) | (df2['ANC4 Fetal Heart Rate'].str.contains('quipm')))].fillna('No Equipment', inplace = True)
						 

In [20]:

# Fill in the missing values in the 'ANC Dates' columns to -1 
for i in [1,2,3,4]:
    df2['ANC'+str(i)+' Date'] = df2['ANC'+str(i)+' Date'].map(lambda x: x if x.year>1970 else -1)
	
# Fill in the missing values in ANC Weights with NaN values for the time being (not -1, as next we find out the diff in weights across all ANC's)
for i in ['ANC1 Weight', 'ANC2 Weight', 'ANC3 Weight', 'ANC4 Weight']:
    df2[i] = df2[i].map(lambda x: x if (type(x) == int or type(x) == float) else np.nan)

In [21]:

# Normally the weights tend to increase as the ANC's progress. Although some entries buck this trend, this trend has been assumed here
# to find out the max weight change from the first ANC. If all weight values are missing, set weight change to 0
df2['Max Change in ANC Weight'] = df2[['ANC1 Weight', 'ANC2 Weight', 'ANC3 Weight', 'ANC4 Weight']].apply(lambda x : x.max() - x['ANC1 Weight'] if x['ANC1 Weight'] != -1 else 0, axis = 1)

# Drop the columns as data has been extracted
df2.drop(['ANC1 Weight', 'ANC2 Weight', 'ANC3 Weight', 'ANC4 Weight'], axis = 1, inplace = True)

In [22]:
# Find out the pressure differential (('Sys BP' - 'Dia BP)/'Sys BP') that takes into account the absolute values of the 'Sys BP',
# rather than just the difference in pressure. This is done on the huge dataframe where the large no. of columns
# were not dropped. 
df['ANC4 Pressure Difference'] = df.apply(lambda x : (x['ANC4 BP Sys'] - x['ANC4 BP Dia'])/x['ANC4 BP Sys'] if (type(x['ANC4 BP Sys']) == int and x['ANC4 BP Sys' ] != 0) else 0, axis = 1)
df['ANC3 Pressure Difference'] = df.apply(lambda x : (x['ANC3 BP Sys'] - x['ANC3 BP Dia'])/x['ANC3 BP Sys'] if (type(x['ANC3 BP Sys']) == int and x['ANC3 BP Sys' ] != 0) else 0, axis = 1)
df['ANC2 Pressure Difference'] = df.apply(lambda x : (x['ANC2 BP Sys '] - x['ANC2 BP Dia'])/x['ANC2 BP Sys '] if (type(x['ANC2 BP Sys ']) == int and x['ANC2 BP Sys ' ] != 0) else 0, axis = 1)
df['ANC1 Pressure Difference'] = df.apply(lambda x : (x['ANC1 BP Sys'] - x['ANC1 BP Dia'])/x['ANC1 BP Sys'] if (type(x['ANC1 BP Sys']) == int and x['ANC1 BP Sys' ] != 0) else 0, axis = 1)

In [23]:
# make those values into new columns in our main dataframe replacing -1 values, if any, to 0 
for i in ['ANC1 Pressure Difference', 'ANC2 Pressure Difference', 'ANC3 Pressure Difference', 'ANC4 Pressure Difference']:
    df2[i] = df[i].map(lambda x: 0 if x == -1 else x)
	
# Drop the BP columns as relevant data has been extracted from it.
df2.drop(df2.columns[df2.columns.str.contains('BP')], axis = 1, inplace = True)

# Keep only the ANC1 Hb values as all the other contain too much missing data and fill missing values in ANC1 Hb to -1
df2.drop(['ANC2 HB', 'ANC3 HB', 'ANC4 HB'], axis = 1, inplace = True)
df2['ANC1 HB'] = df2['ANC1 HB'].map(lambda x: -1 if type(x) == str else x).fillna(-1)

# Drop the fundal height valriables as they appear in the risk symptoms engineered features later on.
df2.drop(['ANC1 Fundal Height', 'ANC2 Fundal Height', 'ANC3 Fundal Height', 'ANC4 Fundal Height'], axis = 1, inplace = True)

In [24]:
# Assign values for 'Abnormal', 'Normal', and 'Not Done' values in ANC fetal movements variable 
for i in ['ANC1 Fetal Movements', 'ANC2 Fetal Movements', 'ANC3 Fetal Movements', 'ANC4 Fetal Movements']:
    df2[i] = df[i].map({'Abnormal' : -100, 'Normal' : 100, 'Not Done' : -1})

# Here, we find out the max values (100) and min values (-100) along the different Fetal mov variables in one entry, temporary feature.
df2['max'] = df2[['ANC1 Fetal Movements', 'ANC2 Fetal Movements', 'ANC3 Fetal Movements', 'ANC4 Fetal Movements']].max(axis= 1)
df2['min'] = df2[['ANC1 Fetal Movements', 'ANC2 Fetal Movements', 'ANC3 Fetal Movements', 'ANC4 Fetal Movements']].min(axis= 1)
 
# Create a new feature that shows whether fetal mov was normal or abnormal
# If along the row, the max value of 100 is present, then fetal mov was normal in atleast one ANC checkup, so assign 1 to it (normal)
# Else assign zero (abnormal)
df2['Fetal Movement Nomal/Abnormal'] = df2.apply(lambda x: 1 if x['max'] == 100 else 0, axis = 1)

# If both max and min are -1 (missing), assign to the new feature a value of 2, indicating 'Unknown' 
df2.loc[((df2['max'] == -1) & (df2['min'] == -1)), 'Fetal Movement Nomal/Abnormal'] = 2

#Drop temp columns max and min
df2.drop(['max', 'min'], axis = 1, inplace = True)